**Modification:**


In [2]:
import pandas as pd
import numpy as np
import pickle
from utils import * 
from attention_lstm import *

Using Theano backend.


In [9]:
from keras.models import Sequential, Model
from keras.layers.core import Dense
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.layers import Input, Reshape, Flatten, merge, Activation
from keras.layers.wrappers import Bidirectional
from keras.utils.data_utils import get_file
from keras.regularizers import WeightRegularizer
from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from theano.ifelse import ifelse
from keras.layers.wrappers import TimeDistributed

In [10]:
with open('../input/parameters.pkl', "rb") as f:
    maxlen = pickle.load(f)
    num_features = pickle.load(f)
    chartoindice = pickle.load(f)
    targetchartoindice = pickle.load(f)
    divisor = pickle.load(f)
    divisor2 = pickle.load(f)

In [11]:
with open('../input/preprocessed_data.pkl', "rb") as f:
    X = pickle.load(f)
    y_a = pickle.load(f)
    y_t = pickle.load(f)
    X_test = pickle.load(f)
    y_a_test = pickle.load(f)
    y_t_test = pickle.load(f)

In [12]:
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializations

In [13]:
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializations.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(** kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):
        eij = K.tanh(K.dot(x, self.W))

        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')

        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [15]:
# build the model: 
print('Build model...')
main_input = Input(shape=(maxlen, num_features), name='main_input')


# one shared layer
l1 = LSTM(100, consume_less='gpu', init='glorot_uniform', return_sequences=True, dropout_W=0.2)(main_input)
b1 = BatchNormalization()(l1)



# the layer specialized in activity prediction
l2_1 = LSTM(100, consume_less='gpu', init='glorot_uniform', return_sequences=False, dropout_W=0.2)(b1) 
b2_1 = BatchNormalization()(l2_1)
a = Activation('softmax')(b2_1)

# the layer specialized in time prediction
l2_2 = LSTM(100, consume_less='gpu', init='glorot_uniform', return_sequences=False, dropout_W=0.2)(l1) 
b2_2 = BatchNormalization()(l2_2)

# last activation
act_output = Dense(len(targetchartoindice), activation='softmax', init='glorot_uniform', name='act_output')(a)
time_output = Dense(1, init='glorot_uniform', name='time_output')(b2_2)


model = Model(input=[main_input], output=[act_output, time_output])

opt = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004, clipvalue=3)

model.compile(loss={'act_output':'categorical_crossentropy', 'time_output':'mae'}, optimizer=opt)
early_stopping = EarlyStopping(monitor='val_loss', patience=42)
model_checkpoint = ModelCheckpoint('output_files/models/model_{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)

model.fit(X, {'act_output':y_a, 'time_output':y_t}, validation_split=0.2, verbose=2, callbacks=[early_stopping, model_checkpoint, lr_reducer], batch_size=maxlen, nb_epoch=500)

Build model...


/Users/hoangnguyen/miniconda3/envs/pydata2/lib/python2.7/site-packages/theano/tensor/basic.py:5130: UserWarning: flatten outdim parameter is deprecated, use ndim instead.
  "flatten outdim parameter is deprecated, use ndim instead.")


Train on 7344 samples, validate on 1837 samples
Epoch 1/500
25s - loss: 2.6167 - act_output_loss: 1.5167 - time_output_loss: 1.1000 - val_loss: 2.0994 - val_act_output_loss: 1.0663 - val_time_output_loss: 1.0332
Epoch 2/500
25s - loss: 1.8639 - act_output_loss: 0.8368 - time_output_loss: 1.0271 - val_loss: 1.7412 - val_act_output_loss: 0.6840 - val_time_output_loss: 1.0571
Epoch 3/500
24s - loss: 1.7381 - act_output_loss: 0.7234 - time_output_loss: 1.0147 - val_loss: 1.6095 - val_act_output_loss: 0.6146 - val_time_output_loss: 0.9949
Epoch 4/500
24s - loss: 1.6826 - act_output_loss: 0.6777 - time_output_loss: 1.0049 - val_loss: 1.5944 - val_act_output_loss: 0.6083 - val_time_output_loss: 0.9860
Epoch 5/500
25s - loss: 1.6533 - act_output_loss: 0.6555 - time_output_loss: 0.9978 - val_loss: 1.5905 - val_act_output_loss: 0.5934 - val_time_output_loss: 0.9971
Epoch 6/500
25s - loss: 1.6517 - act_output_loss: 0.6549 - time_output_loss: 0.9968 - val_loss: 1.5583 - val_act_output_loss: 0.5858

KeyboardInterrupt: 

In [16]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
main_input (InputLayer)          (None, 15, 14)        0                                            
____________________________________________________________________________________________________
lstm_6 (LSTM)                    (None, 15, 100)       46000       main_input[0][0]                 
____________________________________________________________________________________________________
batchnormalization_6 (BatchNorma (None, 15, 100)       400         lstm_6[0][0]                     
____________________________________________________________________________________________________
lstm_7 (LSTM)                    (None, 100)           80400       batchnormalization_6[0][0]       
___________________________________________________________________________________________